In [ ]:
# This reloads python code imported from *.py files automatically
%reload_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tqdm 
import tqdm.notebook

from optimization import *

import time
# optimal solution
from optimal_solution import opt_solution

In [ ]:
# Uniform distribution

g1 = G1()
# Plot a histogram of the v array
plt.hist(g1.v, bins=30)
plt.show()

# plot the position of the points
plt.figure(figsize=(5,5))
plt.scatter(g1.x[:,0], g1.x[:,1])
plt.show()

g1.refresh() # generate a new dataset
plt.hist(g1.v, bins=30)
plt.show()

m = np.array([0., 0.])

for _ in range(10):
  g1.refresh() # refresh the dataset
  m  += 0.1*g1.x.mean()

print(m)

# Log normal distribution
g2 = G2()
plt.hist(g2.v, bins=30)
plt.show()
g2.refresh()
plt.hist(g2.v, bins=30)
plt.show()

---

In [ ]:
np.random.seed(42)
#n_iter = 20000
n_iter = 5000

# parametersc
l = 0.8 # lambda in [0,1]
#l = 0.2 # lambda in [0,1]
beta = 25
#beta = 0.7
beta=100

# initialization
N = 1000
n_selected = 0
g = G1(N)
g = G2(N)
citiesX = g.x
citiesV = g.v

#other options
initial_selection_probability=0.2
#initial_selection_probability=1/N


## Optimizations with different stategies

In [ ]:
def plotResult(duration,selected_cities_n,selected_cities_n_convex,loss_values,loss_value_convex,num_cities_per_step):
    print("d= %s seconds" % duration)
    if loss_value_convex is not None:
        print('Final loss '+ str(loss_values[-2])) #be careful, at the end we want to return '-final loss'
        print('Final loss with Convex Hull '+ str(loss_value_convex))
    else:
        print('Final loss '+ str(loss_values[-1]))
    #print(selected_cities_n)

    fig,axes=plt.subplots(1,2,figsize=(12,4))
    #fig.suptitle('Results')
    axes[0].plot(loss_values)
    axes[0].set_ylabel('Loss')
    axes[0].set_xlabel('Iterations')
    axes[0].set_title('Loss Evolution')
    m = selected_cities_n == 1
    axes[1].scatter(g.x[:, 0], g.x[:, 1],label='Non selected Cities')
    axes[1].scatter(g.x[m, 0], g.x[m, 1], c='r',label='Selected cities')
    if selected_cities_n_convex is not None:
        mbis = (selected_cities_n_convex==1) & (selected_cities_n==0)
        mter = (selected_cities_n_convex==0) & (selected_cities_n==1)
        axes[1].scatter(g.x[mbis, 0], g.x[mbis, 1], c='g',label='Added cities (Convex Hull)')
        axes[1].scatter(g.x[mter, 0], g.x[mter, 1], c='y',label='Selected Cities not in Convex Hull')
    axes[1].set_title('Selected cities')
    box = axes[1].get_position()
    axes[1].set_position([box.x0, box.y0, box.width * 0.8, box.height])
    # Put a legend to the right of the current axis
    axes[1].legend(loc='center left', bbox_to_anchor=(1, 0.5))
        
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    
    if num_cities_per_step is not None:
        plt.figure(figsize=(4,2))
        plt.plot(np.arange(n_iter), num_cities_per_step)
        plt.title("#selected cities in each step")

#### Optimization with mutation strategy 0

In [ ]:
# Run the optimization to compute the selected cities
np.random.seed(52)
start_time = time.time()
selected_cities_n, selected_cities_n_convex, loss_values,loss_value_convex = optimize(g, l, beta=beta, n_iter=n_iter,mutation_strategy=0, initial_selection_probability=initial_selection_probability,precompute_pairwise_dist=False, verbose=True)
d=(time.time() - start_time)
num_cities_per_step = np.array(selected_cities_n).sum(axis=1)
if type(selected_cities_n) == list:
    selected_cities_n = selected_cities_n[-1]
    
loss_values[-1]=(loss_value_convex) 
plotResult(d,selected_cities_n,selected_cities_n_convex,loss_values,loss_value_convex,num_cities_per_step)

# f_st.append(loss_values)
# times_st.append(d)


#### Optimization with mutation strategy 2 = Flipping strategy

In [ ]:
np.random.seed(52)
start_time = time.time()
selected_cities_n, selected_cities_n_convex, loss_values,loss_value_convex = optimize(g, l, beta=beta, 
                                          n_iter=n_iter,mutation_strategy=2,initial_selection_probability=initial_selection_probability,
                                          precompute_pairwise_dist=False, verbose=True)
d=(time.time() - start_time)
if type(selected_cities_n) == list:
    selected_cities_n = selected_cities_n[-1]
    
loss_values[-1]=(loss_value_convex)   
plotResult(d,selected_cities_n,selected_cities_n_convex,loss_values,loss_value_convex,None)  
    
# f_st.append(loss_values)
# times_st.append(d)

#### Optimization with mutation strategy 3 = Convex Hull function

In [ ]:
np.random.seed(52)
start_time = time.time()
selected_cities_n, selected_cities_n_convex, loss_values,loss_value_convex = optimize(g, l, beta=beta, 
                                          n_iter=n_iter,mutation_strategy=3,initial_selection_probability=initial_selection_probability,
                                          precompute_pairwise_dist=False, verbose=True)
d=(time.time() - start_time)
if type(selected_cities_n) == list:
    selected_cities_n = selected_cities_n[-1]

loss_values[-1]=(loss_value_convex)   
plotResult(d,selected_cities_n,selected_cities_n_convex,loss_values,loss_value_convex,None)  

# f_st.append(loss_values)
# times_st.append(d)

### Combination 3 and 2

In [ ]:
# First run
start_time = time.time()
selected_cities_n, selected_cities_n_convex, loss_values1,loss_value_convex = optimize(g, l, beta=beta, 
                                          n_iter=n_iter,mutation_strategy=3,
                                          precompute_pairwise_dist=False, verbose=True)
d1=(time.time() - start_time)
loss_values1[-1]=(loss_value_convex)  
if type(selected_cities_n) == list:
    selected_cities_n = selected_cities_n[-1]

# Second run  
start_time = time.time()
selected_cities_n, selected_cities_n_convex, loss_values,loss_value_convex = optimize_with_initialize(g, l, selected_cities_n_convex, beta=beta, 
                                          n_iter=n_iter,mutation_strategy=2,
                                          precompute_pairwise_dist=False, verbose=True)
d2=(time.time() - start_time)
if type(selected_cities_n) == list:
    selected_cities_n = selected_cities_n[-1]
loss_values[-1]=(loss_value_convex) 

total_loss=np.concatenate((loss_values1,loss_values))
total_loss=total_loss[::2]
plotResult(d1+d2,selected_cities_n,selected_cities_n_convex,total_loss,loss_value_convex,None)  

# f_st.append(loss_values)
# times_st.append(d)

#### Optimization with old code (for time comparison)

In [ ]:
# # Run for comparison
# np.random.seed(52)
# start_time = time.time()
# selected_cities_n, loss_values = optimize_old(g, l, beta=beta, n_iter=n_iter,precompute_pairwise_dist=False, verbose=True)
# d=(time.time() - start_time)
# print("d= %s seconds" % d)
# if type(selected_cities_n) == list:
#     selected_cities_n = selected_cities_n[-1]
# print(selected_cities_n)
# print('Final loss '+ str(loss_values[n_iter-1]))
# plt.figure()
# plt.plot(np.arange(n_iter), loss_values);

# plt.figure()
# m = selected_cities_n == 1
# plt.scatter(g.x[:, 0], g.x[:, 1])
# plt.scatter(g.x[m, 0], g.x[m, 1], c='r')
# plt.xlim(0, 1)
# plt.ylim(0, 1)

# f_st.append(loss_values)
# times_st.append(d)

## Plot Comparisons

In [ ]:
n_iter = 5000

# parameters
l = 0.8 # lambda in [0,1]
beta = 50

# initialization
N = 100
n_selected = 0
g = G1(N)
g = G2(N)
citiesX = g.x
citiesV = g.v

In [ ]:
best_sol, best_f = opt_solution(N, l, g, verbose=True)
print(best_f)

In [ ]:
initial_selection_probability=1/N

In [ ]:
np.random.seed()
selected_cities=(np.random.rand(N) <= initial_selection_probability).astype(np.int32)

In [ ]:
f_st=np.zeros((5,n_iter))
selected_st=np.zeros((5,N))
fig=plt.plot
for i in range (4):
    selected_cities_n, selected_cities_n_convex, loss_values,loss_value_convex = optimize_with_initialize(g, l, selected_cities, beta=beta, 
                                              n_iter=n_iter,mutation_strategy=i, initial_selection_probability=initial_selection_probability,
                                              precompute_pairwise_dist=False, verbose=True)
    if type(selected_cities_n) == list:
        selected_cities_n = selected_cities_n[-1]
    loss_values[-1]=(loss_value_convex) 
    print(loss_value_convex)
    f_st[i]=loss_values
    plt.plot(f_st[i],label = "mutation_strategy="+str(i))
    selected_st[i]=selected_cities_n_convex
    
#combining 3 and 2    
selected_cities_n, selected_cities_n_convex, loss_values1,loss_value_convex = optimize_with_initialize(g, l,selected_cities, beta=beta,  n_iter=n_iter,mutation_strategy=3, precompute_pairwise_dist=False, verbose=True)
loss_values1[-1]=(loss_value_convex)  
if type(selected_cities_n) == list:
    selected_cities_n = selected_cities_n[-1]
selected_cities_n, selected_cities_n_convex, loss_values,loss_value_convex = optimize_with_initialize(g, l, selected_cities_n_convex, beta=beta,n_iter=n_iter,mutation_strategy=2,precompute_pairwise_dist=False, verbose=True)
if type(selected_cities_n) == list:
    selected_cities_n = selected_cities_n[-1]
loss_values[-1]=(loss_value_convex) 
total_loss=np.concatenate((loss_values1,loss_values))
total_loss=total_loss[::2]
f_st[4]=total_loss
selected_st[4]=selected_cities_n_convex
plt.plot(f_st[4],label = "Combining")

plt.hlines(best_f, 0, n_iter, linestyles="--", label="optimal solution")

plt.legend()
print(np.amin(f_st,1))
print(np.amin(f_st))
best_min=np.amin(f_st)
best=np.argmin(np.amin(f_st,1))
print("The best cost is=%i, found for strategy %i"%(best_min,best))

plotResult(0,selected_st[best],None,f_st[best],None,None)


## Play on beta (not finished)

Find initial beta= hight T

In [ ]:
#n_iter = 10000
beta=0.001

np.random.seed(52)
start_time = time.time()
selected_cities_n, selected_cities_n_convex, loss_values,loss_value_convex = optimize(g, l, beta=beta, 
                                          n_iter=n_iter,mutation_strategy=2, precompute_pairwise_dist=False, verbose=True)

if type(selected_cities_n) == list:
    selected_cities_n = selected_cities_n[-1]
print('Final loss '+ str(loss_values[n_iter-1])) #be careful, at the end we want to return '-final loss'
print('Final loss convex '+ str(loss_value_convex))
plt.figure()
plt.plot(np.arange(n_iter), loss_values);

beta=1
start_time = time.time()
selected_cities_n, selected_cities_n_convex, loss_values,loss_value_convex = optimize_with_initialize(g, l, selected_cities_n_convex, beta=beta, 
                                          n_iter=n_iter,mutation_strategy=2,
                                          precompute_pairwise_dist=False, verbose=True,)
if type(selected_cities_n) == list:
    selected_cities_n = selected_cities_n[-1]
print('Final loss '+ str(loss_values[n_iter-1])) #be careful, at the end we want to return '-final loss'
print('Final loss convex '+ str(loss_value_convex))
plt.figure()
plt.plot(np.arange(n_iter), loss_values);

beta=10
start_time = time.time()
selected_cities_n, selected_cities_n_convex, loss_values,loss_value_convex = optimize_with_initialize(g, l, selected_cities_n_convex, beta=beta, 
                                          n_iter=n_iter,mutation_strategy=2,
                                          precompute_pairwise_dist=False, verbose=True,)
if type(selected_cities_n) == list:
    selected_cities_n = selected_cities_n[-1]
print('Final loss '+ str(loss_values[n_iter-1])) #be careful, at the end we want to return '-final loss'
print('Final loss convex '+ str(loss_value_convex))
plt.figure()
plt.plot(np.arange(n_iter), loss_values);

beta=100
start_time = time.time()
selected_cities_n, selected_cities_n_convex, loss_values,loss_value_convex = optimize_with_initialize(g, l, selected_cities_n_convex, beta=beta, 
                                          n_iter=n_iter,mutation_strategy=2,
                                          precompute_pairwise_dist=False, verbose=True,)
d2=(time.time() - start_time)
print("d= %s seconds" % d2)
if type(selected_cities_n) == list:
    selected_cities_n = selected_cities_n[-1]
      
    
#print(selected_cities_n)
print('Final loss '+ str(loss_values[n_iter-1])) #be careful, at the end we want to return '-final loss'
print('Final loss convex '+ str(loss_value_convex))
plt.figure()
plt.plot(np.arange(n_iter), loss_values);

plt.figure()
m = selected_cities_n == 1
mbis = (selected_cities_n_convex==1) & (selected_cities_n==0)
mter = (selected_cities_n_convex==0) & (selected_cities_n==1)
plt.scatter(g.x[:, 0], g.x[:, 1])
plt.scatter(g.x[m, 0], g.x[m, 1], c='r')
plt.scatter(g.x[mbis, 0], g.x[mbis, 1], c='g')
plt.scatter(g.x[mter, 0], g.x[mter, 1], c='y')
plt.xlim(0, 1)
plt.ylim(0, 1)

# loss_values[-1]=(loss_value_convex)
# f_st.append(loss_values)
# times_st.append(d)

**Discussion:**
* convergence in terms of beta ?
* Choice of beta? Small then large, or use the optimal beta of the course (need precomputation?)
* Run the simulation many times? Then pick the min? Or do simulated annealing (with different beta)

**To be added:**
* statistics and visualization (of the cities at their location, with a color corresponding to selected or not)


In [ ]:
a = np.random.rand(10)
idx = np.zeros_like(a)
idx[1] =1
idx[5] = 1




In [ ]:
import scipy
from optimization import *

a = np.arange(16).reshape(4, 4)
a = a.T @ a
print(a)
for i in range(4):
    a[i, i] = 0
scipy.spatial.distance.squareform(a)

for i in range(6):
    print(condensed_to_square(i, 4))

